In [ ]:
##!/usr/bin/env python
"""plot_landuse_WRF.py

Purpose: Plot the dominant land use

Author: Annette L Hirsch @ CLEX, UNSW. Sydney (Australia)
email: a.hirsch@unsw.edu.au
Created: Tue Jul 21 09:46:20 AEST 2020

"""

In [ ]:
# Load packages

import numpy as np
import netCDF4 as nc
import sys
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
#from mpl_toolkits.basemap import Basemap
import matplotlib.colors as colors
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from matplotlib import cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy.crs as ccrs

Define AWS locations

In [ ]:
awsdir = '/g/data/w97/azh561/WRF/obs/AWS_1mindata_20stations'
awsnum = ['066037','066137','066194','067105','067108','067113','061078','061366','066062','067119','068228']
awsnm = ['Sydney Airport','Bankstown','Canterbury','Richmond','Badgerys Creek','Penrith','Williamtown','Norah Head','Observatory Hill','Horsley Park','Bellambi']
awslat = [-33.9465,-33.9176,-33.9057,-33.6004,-33.8969,-33.7195,-32.7939,-33.2814,-33.8607,-33.851,-34.3691]
awslon = [151.1731,150.9837,151.1134,150.7761,150.7281,150.6783,151.8364,151.5766,151.2050,150.8567,150.9291]
naws = len(awsnum)

Define inputs

In [ ]:
datadir = '/g/data/w97/azh561/WRF/sydney800m/'
ndom = 2
EXPNM = "Sydney_800m"

In [ ]:
for dd in range(ndom):
    
    # Read data
    f = nc.Dataset('%sgeo_em.d0%s.nc'%(datadir,dd+1))
    lu = f.variables['LU_INDEX'][0,10:-11,10:-11]
    lat2d = f.variables['XLAT_M'][0,10:-11,10:-11]
    lontmp = f.variables['XLONG_M'][0,10:-11,10:-11]
    lon2d = np.where(lontmp<0.0,lontmp+360,lontmp)
    clon = f.getncattr('CEN_LON')
    nlu = f.getncattr('NUM_LAND_CAT')
    iswater = f.getncattr('ISWATER')
    f.close()
    
    # Mask the water bodies
    luma = np.ma.masked_where(lu == iswater, lu)
    
    # Define region extents
    lonN = np.min(lon2d) +1
    lonX = np.max(lon2d) -1
    latN = np.min(lat2d) +1
    latX = np.max(lat2d) -1
    
    # Figure formatting
    plt.rcParams['savefig.dpi']=250
    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"
    
    # Set up projection
    plotcrs = ccrs.PlateCarree(central_longitude=clon)
    
    gs = mpl.gridspec.GridSpec(nrows=1,ncols=1)
    fig = plt.figure(figsize=(15.0,15.0))
    ax = fig.add_subplot(gs[0,0],projection=plotcrs)

    # Specify the colormap with 1-30 greens and 30-40 as reds
    vmin,vmax = 1,nlu
    colors1 = plt.cm.Greens(np.linspace(0.2,1,30))
    colors2 = plt.cm.Reds(np.linspace(0.2,1,10))
    colors = np.vstack((colors1, colors2))
    import matplotlib.colors as mcolors
    mymap = mcolors.LinearSegmentedColormap.from_list('my_colormap', colors)
    mymap.set_bad(color='lightblue')
    #bounds = MaxNLocator(nbins=(nlu+1)).tick_values(vmin,vmax)
    #norm = BoundaryNorm(boundaries=bounds, ncolors=mymap.N)
    #norm = TwoInnerPointsNormalize(vmin=vmin, vmax=vmax, low=1, up=30)
    
    # Plot the land use
    cm = ax.pcolormesh(lon2d,lat2d,luma,vmin=vmax,vmax=vmin,cmap=mymap,transform=ccrs.PlateCarree())
#    cm = ax.pcolormesh(lon2d,lat2d,luma,vmin=vmax,vmax=vmin,cmap=mymap,norm=norm,transform=ccrs.PlateCarree())
    #ax.set_extent([lonN,lonX,latN,latX], ccrs.PlateCarree())
    ax.coastlines(resolution='10m', color='black', linewidth=1)    
    plt.colorbar(cm, ticks=np.arange(vmin,vmax), orientation='vertical',shrink = 0.5)
    
    # For the inner domain at the station locations
    if dd == 1:
        hloc = ['left','right','left','right','right','right','left','left','left','right','left']
        vloc = ['top','top','bottom','bottom','top','bottom','bottom','bottom','bottom','bottom','bottom']
        for ss in range(naws):
            ax.plot(awslon[ss],awslat[ss],color="black",marker = 'o',transform=ccrs.PlateCarree())

            transform = ccrs.PlateCarree()._as_mpl_transform(ax)
            ax.annotate(awsnm[ss],xy=(awslon[ss],awslat[ss]), xycoords=transform,ha=hloc[ss], va=vloc[ss])
    

    # Save the figure
    plt.savefig("Dominant_Land_Use_d0%s_%s.png" %(dd+1,EXPNM))
    plt.close(fig)


In [ ]:

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap


In [ ]:
for dd in range(ndom):
    
    # Read data
    f = nc.Dataset('%sgeo_em.d0%s.nc'%(datadir,dd+1))
    topo = f.variables['HGT_M'][0,10:-11,10:-11]
    mask = f.variables['LANDMASK'][0,10:-11,10:-11]
    lat2d = f.variables['XLAT_M'][0,10:-11,10:-11]
    lontmp = f.variables['XLONG_M'][0,10:-11,10:-11]
    lon2d = np.where(lontmp<0.0,lontmp+360,lontmp)
    clon = f.getncattr('CEN_LON')
    nlu = f.getncattr('NUM_LAND_CAT')
    iswater = f.getncattr('ISWATER')
    f.close()
    
    # Mask the water bodies
    topoma = np.ma.masked_where(mask == 0, topo)
    
    # Define region extents
    lonN = np.min(lon2d) +1
    lonX = np.max(lon2d) -1
    latN = np.min(lat2d) +1
    latX = np.max(lat2d) -1
    
    # Figure formatting
    plt.rcParams['savefig.dpi']=250
    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"
    
    # Set up projection
    plotcrs = ccrs.PlateCarree(central_longitude=clon)
    
    gs = mpl.gridspec.GridSpec(nrows=1,ncols=1)
    fig = plt.figure(figsize=(15.0,15.0))
    ax = fig.add_subplot(gs[0,0],projection=plotcrs)

    # Specify the colormap 
    cmap = plt.cm.get_cmap('terrain')
    new_cmap = truncate_colormap(cmap, 0.2, 0.9)
    new_cmap.set_bad('blue')
    mn = 0
    mx = 1500
    nbins=15
    levels = MaxNLocator(nbins=nbins).tick_values(mn,mx)
    norm = BoundaryNorm(levels, ncolors=new_cmap.N, clip=True)
    
    # Plot the land use
    cm = ax.pcolormesh(lon2d,lat2d,topoma,vmin=mn,vmax=mx,cmap=new_cmap,norm=norm,transform=ccrs.PlateCarree())
    ax.coastlines(resolution='10m', color='black', linewidth=1)    
    plt.colorbar(cm, ticks=np.arange(mn,mx,100), orientation='vertical',shrink = 0.5)

    # For the inner domain at the station locations
    if dd == 1:
        hloc = ['left','right','left','right','right','right','left','left','left','right','left']
        vloc = ['top','top','bottom','bottom','top','bottom','bottom','bottom','bottom','bottom','bottom']
        for ss in range(naws):
            ax.plot(awslon[ss],awslat[ss],color="black",marker = 'o',transform=ccrs.PlateCarree())

            transform = ccrs.PlateCarree()._as_mpl_transform(ax)
            ax.annotate(awsnm[ss],xy=(awslon[ss],awslat[ss]), xycoords=transform,ha=hloc[ss], va=vloc[ss])
   
    
    # Save the figure
    plt.savefig("Topography_d0%s_%s.png" %(dd+1,EXPNM))
    plt.close(fig)
